In [1]:
import tensorflow
import scipy.io 
import matplotlib.pyplot as plt
import cv2
import keras
from glob import glob
import numpy as np
from tqdm import tqdm
import os
from PIL import Image
import pandas as pd
import cv2

import keras_metrics

from sklearn.model_selection import KFold
# from keras.preprocessing.image import ImageDataGenerator

from keras.applications import mobilenet, resnet50 #, vgg16, inception_v3, resnet50, 
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, History

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

import logging
logging.basicConfig(level=logging.WARNING)
# logging.basicConfig(level=logging.DEBUG)

# from scikitplot.metrics import plot_roc_curve

C:\ProgramData\Miniconda3\envs\pytorch1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
def nhwc2nchw(x, use_n=True):
    if use_n:    i=1
    else:        i=0
    return np.swapaxes(np.swapaxes(x, i, i+2), i+1, i+2)

In [90]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # 10 X conv1d layers
        self.conv_layers = []
        self.conv_layers.append(nn.Conv1d(1, 10, 3, padding=1))
        for i in range(9):
            self.conv_layers.append(nn.Conv1d(10, 10, 3, padding=1))

        # 3 X FC layers
        self.fc_layers = []
        self.fc_layers.append(nn.Linear(400, 64))
        self.fc_layers.append(nn.Linear(64, 32))
        self.fc_layers.append(nn.Linear(32, 2))
            
    def forward(self, x):
        x = np.expand_dims(x, 1)
        x = torch.tensor(x).float()
        
        for i in range(10):
            x = F.relu(x)
            x = self.conv_layers[i](x)

        x = x.view(-1, x.shape[1]*x.shape[2])

        for i in range(len(self.fc_layers)):
            x = F.relu(x)
            x = self.fc_layers[i](x)

        x = F.softmax(x, dim=1)
        return x

net = Net()

In [93]:
for i in net.parameters():
    print(i)

In [91]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

ValueError: optimizer got an empty parameter list

In [85]:
x, y = next(train_generator)

In [86]:
xx = net.forward(x)

In [87]:
xx.shape

torch.Size([10000, 2])

In [88]:
xx

tensor([[0.4869, 0.5131],
        [0.4869, 0.5131],
        [0.4869, 0.5131],
        ...,
        [0.4869, 0.5131],
        [0.4869, 0.5131],
        [0.4869, 0.5131]], grad_fn=<SoftmaxBackward>)

In [94]:
np.empty((0, 5, 5, 5))

array([], shape=(0, 5, 5, 5), dtype=float64)

In [58]:
x = x.astype(np.double)

In [59]:
print(x.shape)
x = np.expand_dims(x, 1)
print(x.shape)

(10000, 40)
(10000, 1, 40)


In [60]:
x = torch.tensor(x).float()

In [61]:
conv_layers = []
conv_layers.append(nn.Conv1d(1, 10, 3, padding=1))
for i in range(9):
    conv_layers.append(nn.Conv1d(10, 10, 3, padding=1))
    
fc_layers = []
fc_layers.append(nn.Linear(400, 64))
fc_layers.append(nn.Linear(64, 32))
fc_layers.append(nn.Linear(32, 2))

In [62]:
for i in range(10):
    x = F.relu(x)
    x = conv_layers[i](x)
    
x = x.view(-1, x.shape[1]*x.shape[2])

for i in range(len(fc_layers)):
    x = F.relu(x)
    x = fc_layers[i](x)
    
x = F.softmax(x, dim=1)

In [63]:
x.shape

torch.Size([10000, 2])

## Params

In [10]:
# all_data_dir = '/media/leetwito/DATA/Datasets/PathoBarIlan/Shlomi2018'
all_data_dir = 'e://Work/PathoBarIlan/Shlomi2018'

is_relative_path_csv = False
seed = 4221

pos_name_init = 'Cancer'
neg_name_init = 'Normal'

use_rgb = False # True=rgb, False=spectral
if use_rgb:
    file_ext = '.png'
else:
    file_ext = '.npy'
    
window_size = (200, 200)
shift = (100, 100)

In [11]:
w,h = window_size
if use_rgb:
    input_shape = (w,h,3)
else:
    input_shape = (w,h,40)
batch_size = 10000

## utils

In [12]:
def read_slide(path):
    mat = scipy.io.loadmat(path)
    spectral = mat["Spec"]
    rgb = mat["Section"]
    shape = rgb.shape
    
    return spectral, rgb

In [13]:
def create_batch_of_crops_from_slide(img, window_size, shift, vis_flag=False):
    crops = []

    n_iter_x = (img.shape[1]-window_size[0])//shift[0] + 1

    n_iter_y = (img.shape[0]-window_size[1])//shift[1] + 1

#     n_iter_x, n_iter_y

    for i in range(n_iter_x):
        for j in range(n_iter_y):
            init_y = i*shift[0]
            init_x = j*shift[1]
        
            crops.append(img[init_x:init_x+window_size[0], init_y:init_y+window_size[1], :])
    if vis_flag:
        visualize_batch_of_crops(crops, n_iter_y, n_iter_x)
    return crops

In [14]:
def visualize_batch_of_crops(crops, n_iter_y, n_iter_x):
    fig, axes = plt.subplots(n_iter_y, n_iter_x, figsize=(5, 5), gridspec_kw = {'wspace':0, 'hspace':0})

    for i in range(n_iter_x):
        for j in range(n_iter_y):
            axes[j, i].imshow(crops[i*n_iter_y + j])
            axes[j, i].axis('off')
            axes[j, i].set_aspect('equal')
    plt.show()

In [15]:
def create_crops_from_fileslist(fileslist, window_size, shift):
    rgb_crops = []
    spectral_crops = []
    labels = []

    for file in tqdm(fileslist):
#         file_name = os.path.basename(file)
#         print('Saving crops for file {} ...'.format(file_name))
#         print(file)
        spectral, rgb = read_slide(file)
        spectral_crops = create_batch_of_crops_from_slide(spectral, window_size=window_size, shift=shift)
        rgb_crops = create_batch_of_crops_from_slide(rgb, window_size=window_size, shift=shift)
        save_dir = file.replace('.mat', '_win{}-{}_shift{}-{}'.format(window_size[0], window_size[1], shift[0], shift[1]))
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        for idx, (im_np, spec_np) in enumerate(zip(rgb_crops, spectral_crops)):
            im = Image.fromarray(im_np)
            im.save(os.path.join(save_dir, '{:05}.png'.format(idx)))
            np.save(os.path.join(save_dir, '{:05}.npy'.format(idx)), spec_np)


In [16]:
def create_crops_from_dir(dir_path, window_size, shift):
    print('Saving crops for slides in dir: {}'.format(dir_path))
    fileslist = glob(dir_path + '/*.mat')
    create_crops_from_fileslist(fileslist, window_size, shift)

In [17]:
def create_csv_for_folder(data_dir, ext):
    if ext[0] == '.':
        ext = ext[1:]
    data_df = pd.DataFrame(columns=['filename', 'label'])
    files = glob(os.path.join(data_dir,'*', '*.{}'.format(ext)))
    files = [file for file in files if "Mixed" not in file]
#     print(data_dir+'/*/*.{}'.format(ext))
    
    init_len = len(data_dir)
    delete_folder = all_data_dir
    if not is_relative_path_csv:
        delete_folder = '/'
    if not delete_folder[-1] == '/':
        delete_folder += '/'
    files = [file.replace(delete_folder, '/') for file in files]
#     print(files)
    labels = [1 if pos_name_init in file else 0 for file in files]
#     print(labels)
    data_df['filename'] = files
    data_df['label'] = labels
#     data_df.to_csv(os.path.join(data_dir, os.path.basename(data_dir)+'.csv'), index=False)
#     print('Created CSV successfully for folder {}'.format(data_dir))
    
    return data_df    

In [18]:
slides = glob(os.path.join(all_data_dir, "*/"))
slides

['e://Work/PathoBarIlan/Shlomi2018\\Case10\\',
 'e://Work/PathoBarIlan/Shlomi2018\\Case11\\',
 'e://Work/PathoBarIlan/Shlomi2018\\Case12\\',
 'e://Work/PathoBarIlan/Shlomi2018\\Case14\\',
 'e://Work/PathoBarIlan/Shlomi2018\\Case16\\',
 'e://Work/PathoBarIlan/Shlomi2018\\Case16b\\',
 'e://Work/PathoBarIlan/Shlomi2018\\Case17\\',
 'e://Work/PathoBarIlan/Shlomi2018\\Case18\\',
 'e://Work/PathoBarIlan/Shlomi2018\\Case19484\\',
 'e://Work/PathoBarIlan/Shlomi2018\\Case8\\']

In [19]:
skf = KFold(n_splits=5, shuffle=True, random_state=seed)

train_slides_all = []
test_slides_all = []
val_slides_all = []

for train_index, test_index in skf.split(np.arange(len(slides)).T, np.arange(len(slides)).T):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_slides_all.append(train_index)
    val_slides_all.append([test_index[0]])
    test_slides_all.append([test_index[1]])

TRAIN: [2 3 4 5 6 7 8 9] TEST: [0 1]
TRAIN: [0 1 2 3 5 7 8 9] TEST: [4 6]
TRAIN: [0 1 3 4 5 6 7 8] TEST: [2 9]
TRAIN: [0 1 2 4 6 7 8 9] TEST: [3 5]
TRAIN: [0 1 2 3 4 5 6 9] TEST: [7 8]


In [20]:
i = 3 # take one of the K-Folds

train_index = train_slides_all[i]
val_index = val_slides_all[i]
test_index = test_slides_all[i]

train_index, val_index, test_index

(array([0, 1, 2, 4, 6, 7, 8, 9]), [3], [5])

In [21]:
def get_dfs_for_indices(slides, index_list):
    dfs = []
    for slide in np.array(slides)[index_list]:
        data_dir = slide
        dfs.append(create_csv_for_folder(data_dir, file_ext))
    df = pd.concat(dfs, ignore_index=True)
    df = df.sample(frac=1, random_state=seed)  # frac=1 is same as shuffling df.
    return df

In [22]:
df_train = get_dfs_for_indices(slides, train_index)
df_test = get_dfs_for_indices(slides, test_index)
df_val = get_dfs_for_indices(slides, val_index)

In [23]:
def get_number_of_batches_in_epoch(df, batch_size):
    input_shape_rgb = (input_shape[0], input_shape[1], 3)
    n=0
    for _, file in df.iterrows():
        if not use_rgb:
            file_path = file[0].replace('.npy', '.png')
        else:
            file_path = file[0]
        img = img_to_array(load_img(file_path, target_size=input_shape_rgb))
        n+=(get_otsu_treshed_img(img)<1).sum()
    return n//batch_size

In [24]:
pd.options.display.max_colwidth = 150

In [25]:
print(len(df_train.index.values))
print(len(set(df_train.index.values)))

print(len(df_train.columns.values))
print(len(set(df_train.columns.values)))

2992
2992
2
2


In [26]:
assert len(set(df_train.label.values)) == 2 and len(set(df_val.label.values)) == 2 and len(set(df_test.label.values)) == 2  

In [27]:
def get_otsu_treshed_img(img):
    assert img.max() > 1
    x = cv2.cvtColor((img).astype(np.uint8), cv2.COLOR_BGR2GRAY)
    x1 = cv2.GaussianBlur(x,(5,5),0)
    ret,thr = cv2.threshold(x1,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#     plot_otsu_triplet(org, threshed)
    return thr

In [28]:
def plot_otsu_triplet(org, thr=None):
    gray = cv2.cvtColor(org, cv2.COLOR_BGR2GRAY)
    plt.figure(figsize=(15, 8))
    plt.subplot(131)
    plt.imshow(cv2.cvtColor(org, cv2.COLOR_BGR2RGB))
    plt.subplot(132)
    plt.imshow(gray, cmap="gray")
    plt.subplot(133)
    if thr is None:
        thr = get_otsu_treshed_img(org)
    plt.imshow(thr, cmap="gray")
    cv2.destroyAllWindows()

####### copying generator_from_df:
https://gist.github.com/timehaven/257eef5b0e2d9e2625a9eb812ca2226b#file-akmtdfgen-py

In [29]:
def sample_norm(X):
#     X = X - X.min()
#     X = X / X.max()
#     X = X - 0.5
    # print(X.min(), X.max()) -> (-0.5, 0.5)
    
#     X = X / 255.
    return X

def generator_pixels_from_df(df, batch_size, shuffle=True): 
    logging.debug("starting iterator")
    while 1: # loop for epochs
        logging.debug("in epoch-while loop")
        if shuffle:
            df = df.sample(frac=1)  # frac=1 is same as shuffling df.

        if use_rgb:
            cur_sample = np.empty([0, 3])
        else:
            cur_sample = np.empty([0, 40])
        
        cur_sample_y = np.empty([0, 1])

        for row in df.iterrows(): # loop for files
            logging.debug("in files-for-loop")
            file = row[1].iloc[0]
            y = row[1].iloc[1]
            
            logging.debug("reading file %s"%file)

            input_shape_rgb = (input_shape[0], input_shape[1], 3)
            if use_rgb:
                rgb_file = file
            else:
                rgb_file = file.replace('.npy', '.png')
                img = np.load(file)
            img_rgb = img_to_array(load_img(rgb_file, target_size=input_shape_rgb))
            if use_rgb:
                img = img_rgb
            threshed = get_otsu_treshed_img(img_rgb).flatten()<1
            if use_rgb:
                img = img.reshape((-1, 3))[threshed]
            else:
                img = img.reshape((-1, 40))[threshed]
                
            img = img/img.max(axis=0)
            logging.debug("n_samples in %d image:"%(len(img)))
        
            cur_sample = np.vstack([cur_sample, img])
            cur_sample_y = np.vstack([cur_sample_y,  np.expand_dims(np.array(len(img)*[y]), 1)])
            logging.debug("read img. before entering batches-while-loop: cur_sample: {}, cur_sample_y: {}".format(str(cur_sample.shape), str(cur_sample_y.shape)))
            
            while len(cur_sample)>batch_size:
                logging.debug("in batches-while-loop: leftover: {} samples".format(len(cur_sample)-batch_size))
                logging.debug("n zeros: " + str(to_categorical(cur_sample_y[:batch_size], num_classes=2)[:, 0].sum()))
                out_x = cur_sample[:batch_size] # np.expand_dims(cur_sample[:batch_size], 1)
                out_y = to_categorical(cur_sample_y[:batch_size], num_classes=2)
                cur_sample = cur_sample[batch_size:]
                cur_sample_y = cur_sample_y[batch_size:]
                logging.debug("shapes: out_x: %d, cur_sample: %d"%(out_x.shape[0], cur_sample.shape[0]))
                
#                 print("n zeros: " + str(to_categorical(cur_sample_y[:batch_size], num_classes=2)[:, 0].sum()))
                
                yield (out_x, out_y)

In [30]:
df_train = df_train[:100]

In [31]:
df_val = df_train
df_test = df_train

In [32]:
STEP_SIZE_TRAIN=get_number_of_batches_in_epoch(df_train, batch_size)
STEP_SIZE_VAL=get_number_of_batches_in_epoch(df_val, batch_size)
# STEP_SIZE_TEST=get_number_of_batches_in_epoch(df_test, batch_size)

In [33]:
train_generator = generator_pixels_from_df(df_train, batch_size)
val_generator = generator_pixels_from_df(df_val, batch_size)
test_generator = generator_pixels_from_df(df_test, batch_size, shuffle=False)

In [ ]:
x, y = next(train_generator)

In [ ]:
x.shape, y.shape

In [ ]:
# input_shape = train_generator.image_shape
# mobilenet_model = mobilenet.MobileNet(include_top=True, weights=None, input_shape=input_shape, classes=2)#, dropout=0.2)
# mobilenet_model = resnet50.ResNet50(include_top=True, weights=None, input_shape=input_shape, classes=2)

In [ ]:
filters = 10
mobilenet_model = keras.Sequential()
if use_rgb:
    inp_shape = 3
else:
    inp_shape = 40

mobilenet_model.add(keras.layers.InputLayer(input_shape=(1, inp_shape)))
mobilenet_model.add(keras.layers.Conv1D(filters, 3, padding='same'))
# mobilenet_model.add(keras.layers.Flatten())

for i in range(10):
    mobilenet_model.add(keras.layers.Conv1D(filters, 3, padding='same'))

mobilenet_model.add(keras.layers.Flatten())
mobilenet_model.add(keras.layers.Dense(2, activation="softmax"))

In [ ]:
mobilenet_model.summary()

In [ ]:
optimizer = Adam(lr=1e-2) # 1e-3
mobilenet_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=[keras_metrics.precision(), keras_metrics.recall()]) #  binary_crossentropy , categorical_crossentropy
# history = History()
lrReduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=4, verbose=1, min_lr=1e-9)
if use_rgb:
    chkpnt = ModelCheckpoint("my_models/model_otsu_rgb_weights_epoch{epoch:02d}-val_loss{val_loss:.3f}.hdf5", save_best_only=True) # -train_loss{history.History()[loss][-1]:.2f}
else:
    chkpnt = ModelCheckpoint("my_models/model_otsu_spec_weights_epoch{epoch:02d}-val_loss{val_loss:.3f}.hdf5", save_best_only=True) # -train_loss{history.History()[loss][-1]:.2f}
num_of_epochs = 100

In [ ]:
history = mobilenet_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VAL,
                    epochs=num_of_epochs, callbacks=[lrReduce, chkpnt], shuffle=False) # chkp

In [ ]:
# STEP_SIZE_TEST=len(df_test)//batch_size
# mobilenet_model.evaluate_generator(train_generator, steps=1)
mobilenet_model.evaluate(xxx, yyy)

In [ ]:
# y_pred = mobilenet_model.predict_generator(train_generator, steps=1)
y_pred = mobilenet_model.predict(xx)
y_pred

In [ ]:
y_pred.argmax(axis=1), y_train.values[:, 1].astype(int)

In [ ]:
y_test = df_test['label'][:len(y_pred)].values

In [ ]:
plot_roc_curve(y_test, y_pred)
plt.show()

In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred.shape

In [ ]:
y_pred.sum()

In [ ]:
y_test.sum()

In [ ]:
(y_pred==y_test).sum()/220